# 2. Retrieval Augmented Generation

In [1]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

pdf extraction: 
* https://pdfminersix.readthedocs.io/en/latest/
* https://github.com/pdfminer/pdfminer.six

```shell
pip install pdfminer.six
```

## 2.1 Document and Data Preprocessing

In [9]:
from langchain_community.document_loaders import PDFMinerPDFasHTMLLoader
from bs4 import BeautifulSoup

loader = PDFMinerPDFasHTMLLoader('newyork.pdf')
docs = loader.load()

In [11]:
print(docs[0].page_content[:300])

<html><head>
<meta http-equiv="Content-Type" content="text/html">
</head><body>
<span style="position:absolute; border: gray 1px solid; left:0px; top:50px; width:612px; height:792px;"></span>
<div style="position:absolute; top:50px;"><a name="1">Page 1</a></div>
<div style="position:absolute; border


In [18]:
import pdfminer
from pdfminer.image import ImageWriter
from pdfminer.high_level import extract_pages

pages = list(extract_pages('newyork.pdf'))

def get_image(layout_object):
    if isinstance(layout_object, pdfminer.layout.LTImage):
        return layout_object
    if isinstance(layout_object, pdfminer.layout.LTContainer):
        for child in layout_object:
            return get_image(child)
    else:
        return None


def save_images_from_page(page: pdfminer.layout.LTPage):
    images = list(filter(bool, map(get_image, page)))
    iw = ImageWriter('images')
    for image in images:
        iw.export_image(image)

for page in enumerate(pages):
    save_images_from_page(page)

In [19]:
len(pages)

8